In [1]:
import sys

sys.path.append("..")


import dreamwalker.models as models

2023-12-23 09:43:59,509 DEBUG MainThread matplotlib matplotlib data path: c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\mpl-data
2023-12-23 09:43:59,512 DEBUG MainThread matplotlib CONFIGDIR=C:\Users\adity\.matplotlib
2023-12-23 09:43:59,512 DEBUG MainThread matplotlib interactive is False
2023-12-23 09:43:59,519 DEBUG MainThread matplotlib platform is win32
2023-12-23 09:43:59,642 DEBUG MainThread matplotlib CACHEDIR=C:\Users\adity\.matplotlib
2023-12-23 09:43:59,650 DEBUG MainThread matplotlib.font_manager Using fontManager instance from C:\Users\adity\.matplotlib\fontlist-v330.json
2023-12-23 09:44:05,197 DEBUG MainThread tensorflow Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2023-12-23 09:44:05,734 DEBUG MainThread h5py._conv Creating converter from 7 to 5
2023-12-23 09:44:05,734 DEBUG MainThread h5py._conv Creating converter from 5 to 7
2023-12-23 09:44:05,738 D

In [7]:
import torch
import torch.nn as nn

In [23]:
from torcheeg.models.transformer.conformer import (
    PatchEmbedding,
    TransformerEncoder,
)

from dreamwalker.models.misc import ResidualStack


# ---CONFORMER EEG ENCODER------------------------------------------------------
class ConformerEEGEncoder(nn.Module):
    def __init__(
        self,
        num_electrodes: int = 128,
        num__samples: int = 500,
        sampling_rate: int = 100,
        embed_dropout: float = 0.5,
        hid_channels: int = 40,
        depth: int = 6,
        heads: int = 10,
        dropout: float = 0.5,
        forward_expansion: int = 4,
        forward_dropout: float = 0.5,
        lat_shape: tuple = (16, 32, 32),
        num_res_blocks: int = 1,
    ):
        super().__init__()
        self.num_electrodes = num_electrodes
        self.num__samples = num__samples
        self.sampling_rate = sampling_rate
        self.embed_dropout = embed_dropout
        self.hid_channels = hid_channels
        self.depth = depth
        self.heads = heads
        self.dropout = dropout
        self.forward_expansion = forward_expansion
        self.forward_dropout = forward_dropout
        self.lat_shape = lat_shape
        self.num_res_blocks = num_res_blocks

        self.emb = PatchEmbedding(num_electrodes, hid_channels, embed_dropout)
        self.enc = TransformerEncoder(
            depth,
            hid_channels,
            heads=heads,
            dropout=dropout,
            forward_expansion=forward_expansion,
            forward_dropout=forward_dropout,
        )

        entr_units = self.get_mock_shape()
        entr_units = entr_units[1] * entr_units[2]
        exit_units = self.lat_shape[0] * self.lat_shape[1] * self.lat_shape[2]
        betw_units = (entr_units + exit_units) // 2

        # Correction FC
        self.cfc = nn.Sequential(
            nn.Linear(entr_units, betw_units),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(betw_units, exit_units),
        )

        # Residual Stack
        self.rst = ResidualStack(
            self.lat_shape[0], self.lat_shape[0], n_residual_blocks=num_res_blocks
        )

    def get_mock_shape(self):
        x = torch.rand(1, 1, self.num_electrodes, self.num__samples)
        x = self.emb(x)
        x = self.enc(x)
        return x.shape

    def forward(self, x):
        x = self.emb(x)
        x = self.enc(x)
        x = x.view(x.shape[0], -1)
        x = self.cfc(x)
        x = x.view(x.shape[0], *self.lat_shape)
        x = self.rst(x)
        return x


In [11]:
c.get_mock_shape()

NameError: name 'c' is not defined